In [49]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from preprocessing import preprocess

In [50]:
data = pd.read_csv("train.csv", index_col=0)
#data_plus = pd.read_csv("training_extra.csv", index_col=0)
#data = pd.concat([data, data_plus], axis=0)

In [51]:
data.tail()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
id,,,,,,,,,,
299995,Adidas,Leather,Small,9.0,No,No,Tote,Blue,12.730812,129.99749
299996,Jansport,Leather,Large,6.0,No,Yes,Tote,Blue,26.633182,19.85819
299997,Puma,Canvas,Large,9.0,Yes,Yes,Backpack,Pink,11.898250,111.41364
299998,Adidas,Nylon,Small,1.0,No,Yes,Tote,Pink,6.175738,115.89080
299999,Under Armour,Canvas,Small,2.0,No,Yes,Backpack,Black,18.568865,26.72762


In [52]:
data = preprocess(data)
data.head()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price,...,VAR_Material,Price_mean_Size,STD_Size,VAR_Size,Price_mean_Style,STD_Style,VAR_Style,Price_mean_Color,STD_Color,VAR_Color
0,Jansport,Leather,Medium,7.0,1.0,0.0,Tote,Black,11.611723,112.15875,...,1528.188830,81.343418,38.707882,1498.300133,81.374865,39.208353,1537.294979,80.513439,39.212778,1537.641978
1,Jansport,Canvas,Small,10.0,1.0,1.0,Messenger,Green,27.078537,68.88056,...,1520.831853,81.424674,38.999298,1520.945266,81.410925,38.868197,1510.736709,82.381308,38.934312,1515.880618
2,Under Armour,Leather,Small,2.0,1.0,0.0,Messenger,Red,16.643760,39.17320,...,1528.188830,81.424674,38.999298,1520.945266,81.410925,38.868197,1510.736709,81.011644,39.391864,1551.718941
3,Nike,Nylon,Small,8.0,1.0,0.0,Messenger,Green,12.937220,80.60793,...,1512.606533,81.424674,38.999298,1520.945266,81.410925,38.868197,1510.736709,82.381308,38.934312,1515.880618
4,Adidas,Canvas,Medium,1.0,1.0,1.0,Messenger,Green,17.749338,86.02312,...,1520.831853,81.343418,38.707882,1498.300133,81.410925,38.868197,1510.736709,82.381308,38.934312,1515.880618


In [53]:
data.dtypes

Brand                            object
Material                         object
Size                             object
Compartments                    float64
Laptop Compartment              float64
Waterproof                      float64
Style                            object
Color                            object
Weight Capacity (kg)            float64
Price                           float64
Laptop Compartment_Missing        int32
Waterproof_Missing                int32
Weight Capacity (kg)_Missing      int32
Price_mean_Brand                float64
STD_Brand                       float64
VAR_Brand                       float64
Price_mean_Material             float64
STD_Material                    float64
VAR_Material                    float64
Price_mean_Size                 float64
STD_Size                        float64
VAR_Size                        float64
Price_mean_Style                float64
STD_Style                       float64
VAR_Style                       float64


---

SANDBOX - testing before implementing in preprocessing function

In [54]:
numerical_columns = data.select_dtypes(include='number').columns
categorical_columns = data.select_dtypes(exclude='number').columns

In [55]:
# don't forget to update columns_to_scale

In [56]:
data.columns

Index(['Brand', 'Material', 'Size', 'Compartments', 'Laptop Compartment',
       'Waterproof', 'Style', 'Color', 'Weight Capacity (kg)', 'Price',
       'Laptop Compartment_Missing', 'Waterproof_Missing',
       'Weight Capacity (kg)_Missing', 'Price_mean_Brand', 'STD_Brand',
       'VAR_Brand', 'Price_mean_Material', 'STD_Material', 'VAR_Material',
       'Price_mean_Size', 'STD_Size', 'VAR_Size', 'Price_mean_Style',
       'STD_Style', 'VAR_Style', 'Price_mean_Color', 'STD_Color', 'VAR_Color'],
      dtype='object')

In [68]:
categorical_columns

Index(['Brand', 'Material', 'Size', 'Style', 'Color'], dtype='object')

In [ ]:
from itertools import product

# Obtenir toutes les combinaisons possibles
combinaisons = list(product(data.columns, data.columns))

# Afficher les combinaisons
print(len(combinaisons))


784


In [ ]:
index_to_del = []
for index, tuple in enumerate(combinaisons, (len(combinaisons) - len(combinaisons))):
    if tuple[0] == tuple[1]:
        index_to_del.append(index)

index_to_del.reverse()

for index in index_to_del:
    del combinaisons[index]

take a look to learn how to optimize it without for loop.

END OF THE SANDBOX

---

In [57]:
print(f"{(len(numerical_columns) + len(categorical_columns))} = {len(data.columns)}")

28 = 28


In [58]:
data.columns

Index(['Brand', 'Material', 'Size', 'Compartments', 'Laptop Compartment',
       'Waterproof', 'Style', 'Color', 'Weight Capacity (kg)', 'Price',
       'Laptop Compartment_Missing', 'Waterproof_Missing',
       'Weight Capacity (kg)_Missing', 'Price_mean_Brand', 'STD_Brand',
       'VAR_Brand', 'Price_mean_Material', 'STD_Material', 'VAR_Material',
       'Price_mean_Size', 'STD_Size', 'VAR_Size', 'Price_mean_Style',
       'STD_Style', 'VAR_Style', 'Price_mean_Color', 'STD_Color', 'VAR_Color'],
      dtype='object')

In [59]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler
)
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_validate

In [60]:
data.columns

Index(['Brand', 'Material', 'Size', 'Compartments', 'Laptop Compartment',
       'Waterproof', 'Style', 'Color', 'Weight Capacity (kg)', 'Price',
       'Laptop Compartment_Missing', 'Waterproof_Missing',
       'Weight Capacity (kg)_Missing', 'Price_mean_Brand', 'STD_Brand',
       'VAR_Brand', 'Price_mean_Material', 'STD_Material', 'VAR_Material',
       'Price_mean_Size', 'STD_Size', 'VAR_Size', 'Price_mean_Style',
       'STD_Style', 'VAR_Style', 'Price_mean_Color', 'STD_Color', 'VAR_Color'],
      dtype='object')

In [61]:
ohe_columns = [
    "Brand",
    "Material",
    "Size",
    #"Compartments",
    "Style",
    "Color"
]

columns_to_scale = [
    "Compartments",
    "Weight Capacity (kg)"
] + [col for col in data.columns if "mean" in col or "Count" in col or "STD" in col or "VAR" in col]

In [62]:
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
std = StandardScaler()

In [63]:
column_transformer = ColumnTransformer(
    transformers=[
        ("OneHotEncoding", ohe, ohe_columns),
        ("Standardization", std, columns_to_scale)
    ],
    remainder="passthrough",
    n_jobs=-1,
    verbose=True,
    verbose_feature_names_out=False
).set_output(transform="pandas")

In [64]:
pipeline = Pipeline(
    [
        ("ColumnTransformer", column_transformer),
    ]
)

In [65]:
X = data.drop(columns='Price')
y = data['Price']

In [66]:
X = pipeline.fit_transform(X)

In [ ]:
X["Laptop Compartment"].unique()

array([ 1.,  0., -1.])

In [ ]:
from sklearn.linear_model import LinearRegression
import xgboost as xgb

In [ ]:
#model = LinearRegression()
model = xgb.XGBRegressor()

---

CROSS VAL

In [ ]:
cv_results = cross_validate(model, X, y, cv=5, verbose=4, return_train_score=True, scoring='neg_root_mean_squared_error', n_jobs=-1)
cv_results

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  2.2min remaining:  3.3min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.3min finished


{'fit_time': array([105.80992913,  90.73137784,  99.36208344,  97.10419178,
         97.74413037]),
 'score_time': array([6.825876  , 3.85525918, 7.00057316, 7.2678504 , 7.34825206]),
 'test_score': array([-38.91631875, -38.85960415, -38.88155717, -38.85792384,
        -38.86833527]),
 'train_score': array([-38.7798673 , -38.79510902, -38.78903986, -38.79542708,
        -38.79228503])}

with standardization on

{'fit_time': array([105.80992913,  90.73137784,  99.36208344,  97.10419178,
         97.74413037]),
 'score_time': array([6.825876  , 3.85525918, 7.00057316, 7.2678504 , 7.34825206]),
 'test_score': array([-38.91631875, -38.85960415, -38.88155717, -38.85792384,
        -38.86833527]),
 'train_score': array([-38.7798673 , -38.79510902, -38.78903986, -38.79542708,
        -38.79228503])}

In [ ]:
model.fit(X, y, verbose=3)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

---

GRID SEARCH

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
xgboost_param_grid = {
    'n_estimators': [100, 200, 300],  
    'max_depth': [6, 10, 15],  
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [0.1, 1.0, 10.0],
    'eval_metric': ['logloss'],
    'booster': ['gbtree'],
    'min_child_weight': [1, 3, 5]
}

xgboost_grid = GridSearchCV(
    estimator=model,
    param_grid=xgboost_param_grid,
    scoring='neg_root_mean_squared_error',
    cv=5,
    verbose=3,
    n_jobs=-1
)

In [ ]:
#xgboost_grid.fit(X, y)
#print(f"Best XGBoost Params: {xgboost_grid.best_params_}")
#xgboost_best_model = xgboost_grid.best_estimator_

---

DATA TEST

In [ ]:
test = pd.read_csv('test.csv', index_col=0)
test.head()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg)
id,,,,,,,,,
300000,Puma,Leather,Small,2.0,No,No,Tote,Green,20.671147
300001,Nike,Canvas,Medium,7.0,No,Yes,Backpack,Green,13.564105
300002,Adidas,Canvas,Large,9.0,No,Yes,Messenger,Blue,11.809799
300003,Adidas,Nylon,Large,1.0,Yes,No,Messenger,Green,18.477036
300004,NaN,Nylon,Large,2.0,Yes,Yes,Tote,Black,9.907953


In [ ]:
test = preprocess(test, submission=True)
test.head()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Laptop Compartment_Missing,...,VAR_Material,Price_mean_Size,STD_Size,VAR_Size,Price_mean_Style,STD_Style,VAR_Style,Price_mean_Color,STD_Color,VAR_Color
id,,,,,,,,,,,,,,,,,,,,,
300000,Puma,Leather,Small,2.0,0.0,0.0,Tote,Green,20.671147,0,...,1513.233576,81.467620,38.886324,1512.146174,81.500007,39.041428,1524.233096,82.252359,38.805116,1505.837029
300001,Nike,Canvas,Medium,7.0,0.0,1.0,Backpack,Green,13.564105,0,...,1513.705730,81.201377,38.594907,1489.566813,81.357717,38.922556,1514.965349,82.252359,38.805116,1505.837029
300002,Adidas,Canvas,Large,9.0,0.0,1.0,Messenger,Blue,11.809799,0,...,1513.705730,81.611747,39.257066,1541.117223,81.185531,38.807942,1506.056367,82.013390,38.814955,1506.600759
300003,Adidas,Nylon,Large,1.0,1.0,0.0,Messenger,Green,18.477036,0,...,1508.982042,81.611747,39.257066,1541.117223,81.185531,38.807942,1506.056367,82.252359,38.805116,1505.837029
300004,Inconnu,Nylon,Large,2.0,1.0,1.0,Tote,Black,9.907953,0,...,1508.982042,81.611747,39.257066,1541.117223,81.500007,39.041428,1524.233096,80.326088,39.050068,1524.907809


In [ ]:
test = pipeline.transform(test)
test.head()

,Brand_Adidas,Brand_Inconnu,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,Material_Canvas,Material_Inconnu,Material_Leather,Material_Nylon,...,STD_Style,VAR_Style,Price_mean_Color,STD_Color,VAR_Color,Laptop Compartment,Waterproof,Laptop Compartment_Missing,Waterproof_Missing,Weight Capacity (kg)_Missing
id,,,,,,,,,,,,,,,,,,,,,
300000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.781157,0.778032,1.371333,-0.673459,-0.675059,0.0,0.0,0,0,0
300001,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.117901,-0.119198,1.371333,-0.673459,-0.675059,0.0,1.0,0,0,0
300002,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.984752,-0.981696,1.003200,-0.621579,-0.623321,0.0,1.0,0,0,0
300003,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.984752,-0.981696,1.371333,-0.673459,-0.675059,1.0,0.0,0,0,0
300004,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.781157,0.778032,-1.596095,0.618114,0.616854,1.0,1.0,0,0,0


In [ ]:
preds = pd.concat([pd.DataFrame(test.index), pd.DataFrame(model.predict(test))], axis=1, ignore_index=True)
preds.columns = ['id', 'Price']
print(len(preds))
preds.head()

200000


,id,Price
0,300000,80.928345
1,300001,82.807838
2,300002,83.539360
3,300003,80.704300
4,300004,77.974869


In [ ]:
preds.to_csv('submission.csv', index=False)